# Ideal Gas Simulation

### Recreation of Jeffrey Chang's project paper

https://jeffjar.me/files/simulating-ideal-gas.pdf

Molecular dynamics simulation involving the numerical solution to Newton's laws of motion.

Jeffrey Chang identifies three tenets of ideal gas theory which is demonstrated by the simulation:
1. Non-equilibrium states evolve to equilibrium;
2. The gas satisfies the equation of state $PV = Nk_BT$;
3. Fluctuations around equilibrium occur, and decrease with large N.

## Methodology

### Description:
- N classical particles
- Box of equal side length L
- Particles are non-interacting "hard spheres" and collide elastically
- Wall specularly reflects particles

### Implementation

Rather than iterating through each particle for every time-step, an event-based molecular dynamics engine jumps across time-steps until a collision occurs, and this algorithm is repeated for the duration of the simulation run. An event queue keeps track of upcoming collisions, sorted by time of occurrence.

<img src="ideal-gas-design-diagram.png">

In [1]:
import numpy as np
import matplotlib.pyplot as plt

#Program Parameters

N = 10 #Number of particles
L = 100 #Length of box
pos = L*np.random.rand(N, 2) #Array of initial particle positions
pos += (1-pos)//(0.95*L)*(0.05*L)  #Move particles away from boundary
pos -= pos//(0.95*L)*(0.05*L)
vel = np.random.rand(N, 2) #Array of initial particle velocities
mass = np.ones(N) #Particles masses
radius = np.ones(N) #Particle radii


In [21]:
arr1 = [1,2,3,4,5]
arr2 = [6,7,8,9,0]
arr = [*zip(arr1, arr2)]


In [22]:
arr

[(1, 6), (2, 7), (3, 8), (4, 9), (5, 0)]

The **State** object controls simulation time, so it must have a method to evolve time.

The simulation time stays in the **State** object, so calculation of collision time must be facilitated by **State**.

In [ ]:
import numpy as np
from itertools import combinations

In [ ]:


class State():
    #Create State with particle number, box length, timestep, mass, and radius parameters
    #TODO: Change mass and radius parameters from constants to distributions
    #TODO: Calculate timestep in terms of precision (box length/timestep),
    #      where precision is the number of iterations required for a particle to traverse the length of the box at a speed of 1.
    def __init__(self, N, L, delta_t, M, R):
        #Change to constants?
        self.N = N
        self.L = L
        self.timestep = delta_t
        self.statetime = 0.0
        #Create particles
        init_pos = L*np.random.rand(N, 2) #Array of initial particle positions
        init_pos += (1-pos)//(0.95*L)*(0.05*L)  #Move particles away from boundary
        init_pos -= pos//(0.95*L)*(0.05*L)
        init_vel = np.random.rand(N, 2)   
        self.particle_list = [Particle(init_pos[i], init_vel[i], M, R, i) for i in range(N)]
        #Create Event Queue
        event_queue = []
        particle_pairs = combinations(list(range(N)), 2)
        #Particle-Particle Collisions
        for IDs in particle_pairs:
            P1 = self.particle_list[IDs[0]]
            P2 = self.particle_list[IDs[1]]           
            event = Collision((P1, P2))   #Create Collision object
            t = collision_timestep(event)   #Calculate time until collision
            event_queue.append([event, t, IDs]) #Mutable list with IDs required to update event queue
        #Particle-Wall Collisions
        for P in self.particle_list:
            wall = wall_type(P)
            event = BoundaryCollision((P, wall))
            t = collision_timestep(event) #Assuming that we start at t=0
            event_queue.append([event, t, (P.ID,)])
        #Event Queue field
        self.queue = event_queue
        sort_event_queue()

    #NOTE - Boundary Collisions may be implemented simply in the evolve_particle method
    # using the argument of geometric mirrors

    def evolve_particle(self, particle, duration):
        #Move particle
        new_pos = particle.pos + particle.vel*duration
        #Give new position
        return new_pos

    def update_event_queue(self, particle_IDs): #TODO - Vectorise?
        for event_item in self.queue:
            #Find queued events sharing any particle IDs with given argument
            if any(ID in event_item[2] for ID in particle_IDs):
                event_item[1] = self.statetime + collision_timestep(event_item[0]) #Update collision time
        sort_event_queue() # Re-sort event queue

    def sort_event_queue(self):
        self.queue.sort(key=lambda x: x[1]) #Sort by collision times       

    def collision_timestep(self, event):  #NOTE - Infers limit on box length L based on greatest particle radius max(R)
        i = 1 # Initialised at '1' to ensure particle travels AT MINIMUM the required distance
        if isinstance(event, Collision): 
            distance = 3*self.R   #Initialise distance to pass the while condition
            while(distance > 2*self.R):
                #Evolve particle by *i* timesteps               
                distance = np.abs(evolve_particle(event.P[0], i*self.timestep) - evolve_particle(event.P[1].pos, i*self.timestep))
                i += 1

        elif isinstance(event, BoundaryCollision): 
            index = np.argwhere(event.N)[:]
            distance = L/2. #Placeholder to satisfy while condition
            while(self.R < distance < L-self.R):
                #Evolve particle by *i* timesteps in axis given by 'index'
                distance = event.P.pos[index] + evolve_particle(event.P, i*self.timestep)[index]
                i += 1

        return i*self.timestep #Return time elapsed until collision
    
    def wall_type(particle): #TODO - Additional types if particle exactly hits the corner, e.g. "upper-right"
       #Calculate trigonometric angles to each corner
       # Starting at the third quadrant, i.e. lower-left corner
       x = particle.pos[0]
       y = particle.pos[1]
       corners = np.arctan2([-y,-x], [-y,L-x], [L-y,L-x], [L-y,-x])
       #Find trigonometric angle of particle velocity and compare with corner angles
       angle = np.arctan2(particle.vel[1], particle.vel[0])
       if angle <= corners[0]:
           return "left"
       elif angle <= corners[1]:
           return "lower"
       elif angle <= corners[2]:
           return "right"
       elif angle <= corners[3]:
           return "upper"
       else:
           return "left"
    
    def main(self):  # Algorithm for a singe simulation step
        #Evolve simulation time until next event
        event = self.queue[0,0]
        delta_t = self.queue[0,1] - self.statetime
        self.statetime = self.queue[0,1]
        for particle in self.particle_list:
            particle.pos = evolve_particle(particle, delta_t)
        #Initiate current event
        [event.P[i].vel = event.collide()[i] for i in len(event.P)]
        # Update event queue
        update_event_queue(self.queue[0,2])

        



In [15]:
import numpy as np
x=1
y=2
print(np.arctan2(x,y))
print(np.arctan2(x,-y))
print(np.arctan2(-x,-y))
print(np.arctan2(-x,y))
print(np.arctan2(-0.1,-1))

0.4636476090008061
2.677945044588987
-2.677945044588987
-0.4636476090008061
-3.0419240010986313


In [13]:
alist = (1,2,3,4) #NONO
values = (6, "right")
any(item in alist for item in values)

False

In [5]:
from itertools import combinations
pairs = combinations(range(3), 2)
for i in pairs:
    print(i)

(0, 1)
(0, 2)
(1, 2)


In [24]:
#Try Object-oriented approach
class Particle():
    def __init__(self, pos, vel, mass, radius, id):
        self.pos = pos
        self.vel = vel
        self.m = mass
        self.r = radius
        self.ID = id

    def update_vel(self, vel):
        self.vel = vel

    def get_momentum(self):
        p = self.m*self.vel
        return p
           

In [ ]:
class Collision():
    def __init__(self, particles):
        self.P = particles
        #TO time OR NOT TO time?

    def collide(self):
        part1 = self.P[0]
        part2 = self.P[1]
        unit = (part1.pos-part2.pos)/np.abs(part1.pos-part2.pos)
        momentum = -2.0*part1.m*part2.m/(part1.m+part2.m)*np.dot((part1.vel-part2.vel), unit)*unit
        new_vels = [part1.vel+momentum/part1.m, part2.vel-momentum/part2.m]
        return new_vels

    
class BoundaryCollision():
    def __init__(self, particle, wall):
        self.P = [particle]
        self.N = unit_normal(wall)

    def collide(self):
        vel = self.P[0].vel
        new_vel = vel - 2.0*np.dot(vel, self.N)*self.N
        return [new_vel]

    def unit_normal(wall):
        match wall:
            case "upper":
                return np.array([0,-1])
            case "left":
                return np.array([1,0])
            case "lower":
                return np.array([0,1])
            case "right":
                return np.array([-1,0])
            

